In [1]:
# !pip install sentencepiece
# !pip install transformers
# !pip install rich[jupyter]

In [2]:
# !nvidia-smi

In [3]:
data_path = 'cnn_dm_train_processed2.csv'#'wikihow_processed.csv'
import pandas as pd
df = pd.read_csv(data_path)

In [4]:
df=df.drop(columns=['Unnamed: 0'])

In [5]:
df.sample(5)

,text,headline
151232,The largest haul of Viking treasure ever found...,Location searched by enthusiast Derek McLennan...
85574,"By . Amanda Williams . PUBLISHED: . 06:41 EST,...",Rory Sutherland no evidence top class degrees...
88407,Shamed former Tory peer Lord Hanningfield is s...,Lord Hanningfield claimed daily allowance 11 t...
14508,Washington (CNN) -- Hurricanes are rare in Haw...,An incumbent Hawaii governor never had lost a ...
41842,"(CNN Student News) -- September 16, 2011 . Med...",Use the weekly Newsquiz to test your knowledge...


In [6]:
df = df.rename(columns={"headline": "headlines"}, errors="raise")

In [7]:
df["text"] = "summarize: "+df["text"]

In [8]:
df.head()

,text,headlines
0,"summarize: The drama is real for ""Real Housewi...",Apollo Nida appears on the reality TV series w...
1,summarize: (CNN) -- Stalin was to the point. N...,"Stalin to his wife: ""I miss you. ... I m as lo..."
2,summarize: (CNN) -- Cristiano Ronaldo won his ...,Cristiano Ronaldo suffers disappointment on oc...
3,summarize: (CNN) -- Funny thing about fish: Yo...,"Study: Gulf oil exposure causes ""severely malf..."
4,"summarize: Phoenix, Arizona (CNN) -- On Tuesda...","Air crashes are extremely rare, says Bill Haga..."


In [9]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
    """display dataframe in ASCII format"""
    console=Console()
    table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)
    
    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])
    
    console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [10]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [11]:
class YourDataSetClass(Dataset):
    """
    Creating a custom dataset for reading the dataset and 
    loading it into the dataloader to pass it to the neural network for finetuning the model
    """
    def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]
           
    def __len__(self):
        return len(self.target_text)

    def __getitem__(self, index):
        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        #cleaning data so as to ensure data is in string type
        source_text = ' '.join(source_text.split())
        target_text = ' '.join(target_text.split())

        source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()
    
        return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [12]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    """
    Function to be called for training with the parameters passed from main function
    """
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]

        if _%10==0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            #console.print(training_logger)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [13]:
def validate(epoch, tokenizer, model, device, loader):
    """
    Function to evaluate model for predictions
    """
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)
            generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%10==0:
                console.print(f'Completed {_}')
            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [14]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs3/" ):
       
    """
    T5 trainer
    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"]) # pytorch random seed
    np.random.seed(model_params["SEED"]) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    #model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = T5ForConditionalGeneration.from_pretrained('./outputs2/model_files')
    model = model.to(device)
  
    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text,target_text]]
    display_df(dataframe.head(2))

  
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
    train_size = 0.8
    train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
    val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
    val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': model_params["TRAIN_BATCH_SIZE"],
        'shuffle': True,
        'num_workers': 0
        }


    val_params = {
        'batch_size': model_params["VALID_BATCH_SIZE"],
        'shuffle': False,
        'num_workers': 0
        }


    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


    # Training loop
    console.log(f'[Initiating Fine Tuning]...\n')

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)
      
    console.log(f"[Saving Model]...\n")
    #Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)


    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
    console.save_text(os.path.join(output_dir,'logs.txt'))
  
    console.log(f"[Validation Completed.]\n")
    console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
    console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [15]:
model_params={
    "MODEL":"t5-small",            # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":1,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":64,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [16]:
T5Trainer(dataframe=df[:], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs3")

[20:09:29] [Model]: Loading t5-small...                    <ipython-input-14-235040df74db>:13
                                                                                             

[20:09:43] [Data]: Reading data...                         <ipython-input-14-235040df74db>:25
                                                                                             

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|   summarize: The drama is real for "Real    | Apollo Nida appears on the reality TV series|
|   Housewives of Atlanta" co-star Phaedra    | with wife Phaedra Parks . Nida allegedly set|
|  Parks, whose husband faces charges in a    |  up fake companies to steal personal info . |
|fraud scheme that a federal agent says stole |  Feds accuse Nida of stealing millions with |
|   millions of dollars. Apollo Nida, who     |  identify theft . Nida spent five years in  |
|appears on the reality TV series with Parks, |        prison before marrying Parks .       |
|was arrested last week following a year-long |                                             |
| investigation by the U.S. Secret Service,   |                                             |
| according to the criminal complaint. Nida,  |                                             |
| who served five years in a federal prison   |                                             |
| for auto title fraud before marrying Parks  |                                             |
|   in 2009, allegedly set up several fake    |                                             |
|collection companies as part of the scheme,  |                                             |
| Special Agent Alexandre Herrera said in a   |                                             |
|  sworn affidavit. But the only thing the    |                                             |
|   collection companies collected was the    |                                             |
|  personal information of people that Nida   |                                             |
|used to get fraudulent auto loans, according |                                             |
|to the complaint. The break in the case came |                                             |
|  in September, when federal investigators   |                                             |
|   arrested Gayla St. Julein, a woman who    |                                             |
|  "described herself as Nida s  right hand   |                                             |
|b****  in executing the legwork of his fraud |                                             |
| schemes," the complaint said. St. Julien,   |                                             |
|   who previously served prison time for     |                                             |
|identity theft, told agents that Nida "only  |                                             |
|paid her  scraps " for her work, compared to |                                             |
| the amount of money he made from the fraud  |                                             |
|schemes," the filing said. She turned on him |                                             |
|and agreed to let agents record their phone  |                                             |
|    calls, it said. Nida "knowingly and      |                                             |
| willfully execute and attempt to execute a  |                                             |
|    scheme to defraud federally insured      |                                             |
|financial institutions by depositing stolen  |                                             |
|    and fraudulently obtained checks and     |                                             |
|  fraudulently obtained auto loan proceeds   |                                             |
|  into bank accounts opened in the stolen    |                                             |
|  identities of real persons, and conspire   |                                             |
|with others to do so, and knowingly convert  |     

FULL Dataset: (272112, 2)

TRAIN Dataset: (217690, 2)

...

Completed 6800

[03:24:55] [Validation Completed.]                         <ipython-input-14-235040df74db>:94
                                                                                             

[Model] Model saved @ outputs3\model_files

[Validation] Generation on Validation data saved @ outputs3\predictions.csv

[Logs] Logs saved @ outputs3\logs.txt